In [1]:
# importing necessary modules
import pandas as pd

In [2]:
# Loading the test files
patient_monthwise_revenue_test = pd.read_csv('data/patient_monthwise_revenue_test.csv')
physio_diagnosis_test = pd.read_csv('data/physio_diagnosis_test.csv')

/home/pradeep/softs/anaconda3/envs/portea/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Revenue test information
patient_monthwise_revenue_test.info()

In [ ]:
# Diagnosis test information
physio_diagnosis_test.info()

In [3]:
# Cleaning and merging data

# Removing unnecessary data

# FVS - doesnt have proper information?
# diagnosis - duplicate data with 'physio_diagnosis_train'
# may be if it is missing in train_classified for particular id, we need to take the value from here ?
# service_name - since we have service_id
# approx_age - since we have AGE in physio_diagnosis_train
# Unnamed: 18 - not required
# Unamed: 19 - not required
patient_monthwise_revenue_test = patient_monthwise_revenue_test.drop(columns=
                                                                       ['FVS', 'diagnosis','service_name', 'approx_age',
                                                                        'Unnamed: 18', 'Unnamed: 19'])

In [4]:
# gender - it is there in patient_monthwise_revenue_train
# LVD - it is there in patient_monthwise_revenue_train
# Service - we have service_id in patient_monthwise_revenue_train
# Unnamed: 7 - not required
# Unamed: 8 - not required
# Appt_id - since there is no mapping of training data we are loading for analysis
physio_diagnosis_test = physio_diagnosis_test.drop(columns=
                                                     ['gender', 'LVD', 'Unnamed: 6', 'Unnamed: 7', 'Service'])

In [6]:
# Getting the visit_count by doing groupby of patient_id values
# Getting only patient_id
df_pat_id = pd.DataFrame(patient_monthwise_revenue_test['patient_id'])
# Performing Group by -> reset index -> renaming column to 'visit_count'
df_pat_visit_count = df_pat_id.groupby(df_pat_id.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'visit_count'})

In [7]:
# Adding visit count also
md_monthwise_revenue_visit_count_test = pd.merge(patient_monthwise_revenue_test, df_pat_visit_count, on='patient_id')

In [8]:
physio_diagnosis_test = physio_diagnosis_test.rename(columns={'patientId': 'patient_id'})

In [9]:
full_test_raw = pd.merge(md_monthwise_revenue_visit_count_test, physio_diagnosis_test, on='patient_id')

In [10]:
full_test_raw.columns.values

array(['patient_id', 'visit_month_year', 'service_id', 'city', 'ref_type',
       'ref_name', 'ref_source', 'FVD', 'FVM', 'gender', 'LVD', 'brand',
       'visits_required', 'avg_nps', 'visit_count', 'diagnosis', 'AGE'],
      dtype=object)

In [11]:
# New order columns as per the same order as train data
new_cols_order = ['patient_id', 'visit_month_year', 'service_id', 'visit_count', 'city', 'ref_type',
       'ref_name', 'ref_source', 'FVD', 'FVM', 'gender', 'LVD', 'brand',
       'visits_required', 'avg_nps', 'diagnosis', 'AGE']

In [12]:
# Ordering the visit_count in same order as Train data
full_test_raw = full_test_raw[new_cols_order]

In [13]:
# Looking at the new order now, which should be the same order as train data
full_test_raw.columns.values

array(['patient_id', 'visit_month_year', 'service_id', 'visit_count',
       'city', 'ref_type', 'ref_name', 'ref_source', 'FVD', 'FVM',
       'gender', 'LVD', 'brand', 'visits_required', 'avg_nps',
       'diagnosis', 'AGE'], dtype=object)

In [14]:
missing_values_count = full_test_raw.isnull().sum()

In [15]:
missing_values_count

patient_id             0
visit_month_year       0
service_id             0
visit_count            0
city                   0
ref_type               0
ref_name               0
ref_source             0
FVD                    0
FVM                    0
gender                31
LVD                    0
brand                  0
visits_required        0
avg_nps                0
diagnosis           1117
AGE                    0
dtype: int64

In [16]:
# Missing gender 31, diagnosis 1117, fill them with 0
full_test_raw = full_test_raw.fillna(0)

In [17]:
cols_to_process = ['visit_month_year','city','ref_type','ref_name',
                     'ref_source','FVD','FVM','gender','LVD','brand',
                     'avg_nps','diagnosis','AGE']

In [18]:
# Converting into string 
full_test_raw[cols_to_process] = full_test_raw[cols_to_process].astype(str)

In [19]:
from six.moves import cPickle as pickle

In [20]:
try:
    # Loading the LabelEncoder dict objects created when training for encoding the Data
    with open('data/cols_with_classes.pickle', 'rb') as f:
        cols_with_classes = pickle.load(f)
except Exception as e:
    print('Exception ', e)

In [21]:
# Encoding the string values using the same encoding we used for encoding during training
for colm in cols_to_process:
    full_test_raw[colm] = cols_with_classes[colm].fit_transform(full_test_raw[colm])

In [ ]:
full_test_raw.info()

In [ ]:
full_test_raw.to_csv('data/full_test_wo_appts_cleaned_encoded.csv', index=False)

In [22]:
try:
    # Loading the classifier
    #with open('data/train_svc_clf.pickle', 'rb') as f:
    with open('data/train_svc_clf_wo_pat_id.pickle', 'rb') as f:
        clf = pickle.load(f)
except Exception as e:
    print('Exception ', e)

In [23]:
# Reading the processed csv file from the disk
full_test_pros = pd.read_csv('data/full_test_wo_appts_cleaned_encoded.csv')

In [24]:
test_pat_ids = full_test_pros['patient_id']

In [26]:
full_test_pros = full_test_pros.drop(columns='patient_id')

In [27]:
# Converting to numpy array
X_test = full_test_pros.as_matrix()

In [28]:
# Running the predictions on the data
Y_test = clf.predict(X_test)

In [29]:
try:
    # Saving the predictions Y_test numpy array
    #with open('data/Y_test_np.pickle', 'wb') as f:
    with open('data/Y_test_np_wo_pat_id.pickle', 'wb') as f:
        pickle.dump(Y_test, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Got exception', e)

In [30]:
try:
    # Loading the Y_test predictions numpy array
    #with open('data/Y_test_np.pickle', 'rb') as f:
    with open('data/Y_test_np_wo_pat_id.pickle', 'rb') as f:
        Y_test_np = pickle.load(f)
except Exception as e:
    print('Exception ', e)

In [31]:
len(Y_test_np)

55329

In [33]:
# full_test_pros is required to add the patient_id to the label
#patient_id_df = pd.DataFrame(full_test_pros['patient_id'])
Y_test_df = pd.DataFrame(Y_test_np)

In [35]:
# Joining the predicted Labels with the patient_id
#predictions = patient_id_df.join(Y_test_df).rename(columns={0: 'Bucket', 'patient_id': 'PID'})
test_pat_ids_df = pd.DataFrame(test_pat_ids)
predictions = test_pat_ids_df.join(Y_test_df).rename(columns={0: 'Bucket', 'patient_id': 'PID'})

In [ ]:
predictions.sample(5)

In [36]:
# Predictions count is more than the submission count
len(predictions)

55329

In [37]:
submission = pd.read_csv('data/Submission.csv')

In [ ]:
for id, row in submission.iterrows():
    submission['Bucket'].iloc[id] = predictions.loc[predictions['PID'] == row['PID'], 'Bucket'].iloc[0]

In [38]:
# Lets see the submission count
len(submission)

32455

Y_test_np/predictions count is more than the submission count

In [39]:
#submission = submission.drop(columns=['Bucket']) # getting error while trying to drop with columns attribute
submission = submission.drop('Bucket', axis=1)

In [40]:
submission.head(5)

,PID
0,111331
1,27477
2,439072
3,346329
4,58664


In [ ]:
submission.columns.values

In [47]:
final_submission = pd.merge(submission, predictions, on='PID')

In [42]:
final_submission.head(5)

,PID,Bucket
0,111331,3
1,27477,3
2,439072,3
3,346329,3
4,58664,3


In [49]:
len(final_submission)

55329

In [44]:
sub_final = final_submission.drop_duplicates()

In [45]:
len(sub_final)

32455

In [46]:
#sub_final.to_csv('data/Final_Submissions.csv', index=False)
sub_final.to_csv('data/Final_Submissions_wo_pat_id.csv', index=False)

In [48]:
final_submission.to_csv('data/Final_Submissions_unprocessed.csv', index=False)

In [27]:
len(final_submission_s_p)

55329

In [28]:
final_submission_s_p.sample(5)

,PID,Bucket
26092,165715,3
46287,338907,3
50556,387707,3
37209,250972,3
47148,347230,3


In [36]:
final_submission_s_p.head(10)

,PID,Bucket
0,122,3
1,122,3
2,122,3
3,122,3
4,122,3
5,122,3
6,122,3
7,122,3
8,122,3
9,122,3


In [29]:
submissioin_final = final_submission_s_p.drop_duplicates()

In [30]:
# Now the count is perfectly fine
len(submissioin_final)

32455

In [31]:
# Checking first few values and last few values
submissioin_final.head(10)

,PID,Bucket
0,122,3
12,129,3
14,10251,3
16,10496,3
30,10701,3
35,10784,3
39,10814,3
69,10875,3
74,10912,3
75,10991,3


In [ ]:
missing_values_count = final_submission.isnull().sum()

In [ ]:
final_submission = final_submission.drop_duplicates()

In [ ]:
len(final_submission)

In [ ]:
final_submission